Студент Жилкин Федор, 344
# Домашняя работа 1: "Точные методы решения СЛАУ."
**Задания:**

1. Метод Гаусса с выбором главного элемента (поиск решения системы)
2. Метод Жордана (поиск обратной матрицы)
3. Метод LU-разложения (вычисление определителя матрицы)
 
$ A = 
\left(\begin{array}{ccc|c} 
6.687233 & 0.80267  & -2.06459 & 0\\ 
0.80267 & 5.07816 & 0.48037  & 1\\
-2.06459 & 0.48037 & 4.02934 & 0
\end{array}\right)
$



In [1]:
import numpy as np

In [2]:
A = np.array([[6.687233, 0.80267, -2.06459], [0.80267, 5.07816, 0.48037], [-2.06459, 0.48037, 4.02934]])

In [3]:
B = np.array([[0.], [1.], [0.]])

## Метод Гаусса

In [4]:
def gauss(A, B):
    
    # Расширенная матрица
    extended_m = np.array(list(map(lambda i: np.append(A[i], B[i]), range(len(B)))))
    
    # Прямой ход
    def forward_trace(extended_m=extended_m):
        n = len(extended_m)
        
        for k in range(0, n):
            
            # print(extended_m)
            
            # получаем столбец
            column = [ abs(row[k:1+k][0]) for row in A[k:]]

            # Номер строки с максимальным элементом в столбце
            idx = column.index(max(column))

            # Перестановка строк
            tmpstr = np.copy(extended_m[idx + k])
            extended_m[idx + k] = extended_m[k]
            extended_m[k] = tmpstr
            
            # print("После перестановки: ")
            # print(extended_m)
            
            tmp = np.copy(extended_m[k][k])
            for i in range(0, n + 1):
                extended_m[k][i] /= tmp
                
            for i in range(k+1, len(A)):
                tmp = np.copy(extended_m[i][k])
                for j in range(k, len(A)+1):
                    extended_m[i][j] -= extended_m[k][j]*tmp
                
        return extended_m
    
    # Обратный ход
    def back_trace(extended_m):

        def sum_x(x, extended_m, i):
            sum = 0
            for j in range(i+1, n):
                sum += extended_m[i][j]*x[j]
            return sum     

        n = len(extended_m)
        x = np.zeros(n)
        l = list(range(0, n))
        l.reverse()

        for i in l:
            x[i] = extended_m[i][n] - sum_x(x, extended_m, i)

        return x 
    
    return back_trace(forward_trace())

In [5]:
gauss(A,B)

array([-0.03860958,  0.20723292, -0.04448903])

**Ответ:**
$ X = 
\begin{pmatrix}
 -0.03860958 \\
 0.20723292 \\
 -0.04448903
\end{pmatrix}
$

**Проверка с библиотечным методом:**

In [7]:
gauss(A,B) == list(map(lambda x: x[0], np.linalg.solve(A, B)))

array([ True,  True,  True])

## Метод Жордана

In [8]:
def jordan(A, B):

    extended_m = np.array(list(map(lambda i: np.append(A[i], B[i]), range(len(B)))))
    n = len(extended_m)

    for k in range(0, n):

        # print(extended_m)

        # получаем столбец
        column = [ abs(row[k:1+k][0]) for row in A[k:]]

        # Номер строки с максимальным элементом в столбце
        idx = column.index(max(column))

        # Перестановка строк
        tmpstr = np.copy(extended_m[idx + k])
        extended_m[idx + k] = extended_m[k]
        extended_m[k] = tmpstr

        # print("После перестановки: ")
        # print(extended_m)

        tmp = np.copy(extended_m[k][k])
        for i in range(0, n + 1):
            extended_m[k][i] /= tmp

        for i in range(0, len(A)):
            if(i != k):
                tmp = np.copy(extended_m[i][k])
                for j in range(k, len(A)+1):
                    extended_m[i][j] -= extended_m[k][j]*tmp
                    
    return np.array([row[-1:][0] for row in extended_m])

In [9]:
jordan(A, B)

array([-0.03860958,  0.20723292, -0.04448903])

**Ответ:**
$ X = 
\begin{pmatrix}
 -0.03860958 \\
 0.20723292 \\
 -0.04448903
\end{pmatrix}
$

**Проверка с библиотечным методом:**

In [10]:
jordan(A,B) == list(map(lambda x: x[0], np.linalg.solve(A, B)))

array([ True,  True,  True])

## Метод LU-разложения

In [11]:
def LU(A):
    
    def sum_l(i, j):
        sum = 0
        for k in range(0, i-1):
            sum += L[j][k]*U[k][i]
        return sum    

    def sum_u(i, j):
        sum = 0
        for k in range(0, i-1):
            sum += L[i][k]*U[k][j]
        return sum 
    
    L = np.zeros((len(A), len(A)))
    U = np.zeros((len(A), len(A)))
    for j in range(0, len(A)):
        U[0][j] = A[0][j]
        L[j][0] = A[j][0] / U[0][0]
    
    for i in range(1, len(A)):
        
        for j in range(i, len(A)):
            U[i][j] = A[i][j] - sum_u(i, j)
        for j in range(i, len(A)):
            L[j][i] = (A[j][i] - sum_l(i, j))/U[i][i]
    
    return L, U

In [12]:
L, U = LU(A)

In [13]:
L

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.12003021,  1.        ,  0.        ],
       [-0.30873606,  0.09459529,  1.        ]])

In [14]:
U

array([[ 6.687233  ,  0.80267   , -2.06459   ],
       [ 0.        ,  5.07816   ,  0.48037   ],
       [ 0.        ,  0.        ,  3.39192661]])